# BLEU score

In [2]:
cd ..

C:\Users\katar\Desktop\0nmt


In [22]:
from pathlib import Path

import numpy as np
import pytorch_lightning as pl
import pytorch_lightning.callbacks as plc
import torch
import wandb
from icecream import ic
from matplotlib import pyplot as plt
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.loggers.wandb import WandbLogger
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.utils import download_from_url, extract_archive
from torchtext.vocab import vocab
from torchtext.data.metrics import bleu_score
from tqdm import tqdm

from zeronmt.datasets import ParallelDataset
from zeronmt.models.datatypes import DimensionSpec, Language, Vectors
from zeronmt.models.seq2seq import Seq2SeqSupervised, Seq2SeqUnsupervised
from zeronmt.vectors import FastTextAligned

In [4]:
VOCAB_SIZE = int(4.5e4)  # we will use only 45K most-frequent words

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
special_tokens = ["<unk>", "<pad>", "<bos>", "<eos>"]
tokenizer = get_tokenizer("basic_english")  # keep it simple

In [5]:
MAPPING_PATH = Path("best_mapping.pth")

In [6]:
mapping = torch.load(MAPPING_PATH)

# We will use vanilla FastText word embeddings and align them with mapping trained before.
src_vecs = FastTextAligned(
    "https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.cs.vec",
    mapping=None,
    special_toks=special_tokens,
    max_vectors=VOCAB_SIZE
)
tgt_vecs = FastTextAligned(
    "https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.pl.vec",
    mapping=mapping,
    special_toks=special_tokens,
    max_vectors=VOCAB_SIZE
)

In [7]:
opensubtitles_filepaths =("OpenSubtitles.cs-pl.cs", "OpenSubtitles.cs-pl.pl")
test_filepaths = ("TED2020.cs-pl.cs", "TED2020.cs-pl.pl")

In [8]:
tgt_vocab = vocab(tgt_vecs.stoi, min_freq=0)
src_vocab = vocab(src_vecs.stoi, min_freq=0)

In [9]:
src_vocab.set_default_index(src_vocab["<unk>"])
tgt_vocab.set_default_index(tgt_vocab["<unk>"])

In [10]:
BATCH_SIZE = 64

# special tokens are prepended, so these indices are the same for both the languages
PAD_IDX = src_vocab["<pad>"]
BOS_IDX = src_vocab["<bos>"]
EOS_IDX = src_vocab["<eos>"]

In [12]:
cd .data

C:\Users\katar\Desktop\0nmt\.data


In [13]:
train_val_data = ParallelDataset(
    *opensubtitles_filepaths,
    src_vocab,
    tgt_vocab,
    tokenizer,
    max_sentence_len=100,
    max_nsentence=1e6
)
test_data = ParallelDataset(
    *test_filepaths,
    src_vocab,
    tgt_vocab,
    tokenizer,
    max_sentence_len=100,
    max_nsentence=1e6
)

In [15]:
cd ..

C:\Users\katar\Desktop\0nmt


In [16]:
ENC_HID_DIM = 64
DEC_HID_DIM = 64
ATTN_DIM = 8
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

model_loaded  = torch.load("model.ckpt", map_location=torch.device('cpu'))

model = Seq2SeqUnsupervised(DEC_DROPOUT,
    ENC_DROPOUT,
    DimensionSpec(
        attention=ATTN_DIM,
        dec_hid=DEC_HID_DIM,
        enc_hid=ENC_HID_DIM,
        nspecial_toks=len(special_tokens),
    ),
    PAD_IDX,
    Vectors(src_vecs, tgt_vecs))

embedding_src.weight
embedding_tgt.weight
encoder.special_toks_embedding.weight
encoder.rnn.weight_ih_l0
encoder.rnn.weight_hh_l0
encoder.rnn.bias_ih_l0
encoder.rnn.bias_hh_l0
encoder.rnn.weight_ih_l0_reverse
encoder.rnn.weight_hh_l0_reverse
encoder.rnn.bias_ih_l0_reverse
encoder.rnn.bias_hh_l0_reverse
encoder.fc.weight
encoder.fc.bias
decoder.attention.attn.weight
decoder.attention.attn.bias
decoder.special_toks_embedding.weight
decoder.rnn.weight_ih_l0
decoder.rnn.weight_hh_l0
decoder.rnn.bias_ih_l0
decoder.rnn.bias_hh_l0
decoder.output_to_src.weight
decoder.output_to_src.bias
decoder.output_to_tgt.weight
decoder.output_to_tgt.bias


In [17]:
model.load_state_dict(model_loaded['state_dict'])

<All keys matched successfully>

In [18]:
def collate_batch(data_batch):
    src_batch, tgt_batch = [], []
    for src_item, tgt_item in data_batch:
        src_batch.append(
            torch.cat(
                [torch.tensor([BOS_IDX]), src_item, torch.tensor([EOS_IDX])], dim=0
            )
        )
        tgt_batch.append(
            torch.cat(
                [torch.tensor([BOS_IDX]), tgt_item, torch.tensor([EOS_IDX])], dim=0
            )
        )
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

In [19]:
train_data, val_data = torch.utils.data.random_split(train_val_data, [0.8, 0.2])

In [20]:
train_dl = DataLoader(
    train_data,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_batch,
    num_workers=0,
)
valid_dl = DataLoader(
    val_data,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_batch,
    num_workers=0,
)
test_dl = DataLoader(
    test_data,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_batch,
    num_workers=0,
)

In [24]:
tgt_, p_tgt_ = [], []
for i, (src_in, tgt_in) in enumerate(tqdm(test_dl)):
    if i >= (3000/64):
        break
    output = model(
        src_in.to(device),
        tgt_in.to(device),
        Language.src,
        Language.tgt,
        teacher_forcing_ratio=0,
    ).cpu()
    torch.set_printoptions(profile="full")
    predicted_tokens = output.argmax(-1)
    for s_snt, t_snt, pred_snt in zip(src_in.t(), tgt_in.t(), predicted_tokens.t()):
        src_sent = list(filter(lambda x: x not in special_tokens, (src_vocab.get_itos()[t] for t in s_snt)))
        pred_sent = list(filter(lambda x: x not in special_tokens, (tgt_vocab.get_itos()[t] for t in pred_snt)))
        tgt_sent = list(filter(lambda x: x not in special_tokens, (tgt_vocab.get_itos()[t] for t in t_snt)))
        #ic(src_sent)
        #ic(pred_sent)
        #ic(tgt_sent)
        #print()
        tgt_.append([tgt_sent])
        p_tgt_.append(pred_sent)

  2%|█████                                                                                                                                                                                                                                                          | 47/2359 [21:26<17:34:42, 27.37s/it]


In [25]:
bleu_score(p_tgt_, tgt_)

0.006097215227782726

In [28]:
tgt_, p_tgt_ = [], []
for i, (src_in, tgt_in) in enumerate(tqdm(test_dl)):
    if i ==1:
        break
    output = model(
        src_in.to(device),
        tgt_in.to(device),
        Language.src,
        Language.tgt,
        teacher_forcing_ratio=0,
    ).cpu()
    torch.set_printoptions(profile="full")
    predicted_tokens = output.argmax(-1)
    for s_snt, t_snt, pred_snt in zip(src_in.t(), tgt_in.t(), predicted_tokens.t()):
        src_sent = ' '.join(list(filter(lambda x: x not in special_tokens, (src_vocab.get_itos()[t] for t in s_snt))))
        pred_sent = ' '.join(list(filter(lambda x: x not in special_tokens, (tgt_vocab.get_itos()[t] for t in pred_snt))))
        tgt_sent = ' '.join(list(filter(lambda x: x not in special_tokens, (tgt_vocab.get_itos()[t] for t in t_snt))))
        ic(src_sent)
        ic(pred_sent)
        ic(tgt_sent)
        print()
        tgt_.append([tgt_sent])
        p_tgt_.append(pred_sent)

  0%|                                                                                                                                                                                                                                                                           | 0/2359 [00:00<?, ?it/s]ic| src_sent: 'velice ti děkuji .'
ic| pred_sent: 'tu być proszę .'
ic| tgt_sent: 'bardzo dziękuję , chris .'


ic| src_sent: 'je to opravdu veliká čest moci na tomto pódiu vystoupit dvakrát . jsem za to .'
ic| pred_sent: ('jest jest naprawdę sprzedał naszą energię na jego drużyny . . . mnie jeszcze '
                'to .')
ic| tgt_sent: ('to prawdziwy mieć możliwość drugi raz stanąć w tym miejscu . jestem '
               'niezwykle wdzięczny .')


ic| src_sent: ('jsem z této konference nadšený a chtěl bych vám všem za množství komentářů k '
               'mému předchozímu vystoupení .')
ic| pred_sent: ('od mnie tę grę książki i najlepszy byś musimy częściej już rolę oka ci tutaj '
                'żadnych kłopoty .')
ic| tgt_sent: ('jestem tą . chce wam wszystkim za komentarze dotyczące mojej wypowiedzi '
               'poprzedniego wieczoru .')


ic| src_sent: 'a myslím to upřímně , částečně protože – ( falešný ) – to potřebuji !'
ic| pred_sent: 'i wiesz to , , choć starsza ( ( a to bogiem ?'
ic| tgt_sent: 'mówię to szczerze , po części ponieważ ( ) jest mi to potrzebne .'


ic| src_sent: '( smích ) se do mé situace !'
ic| pred_sent: '- ok , się ją chodzi sprowadza ?'
ic| tgt_sent: '( śmiech ) się na moim miejscu !'


...

ic| src_sent: 'jsou to investice , které se .'
ic| pred_sent: 'ten jest zamknięte , który się .'
ic| tgt_sent: 'to są inwestycje , które się zwracają .'


ic| src_sent: 'ale také nás velmi efektivně od naší cesty .'
ic| pred_sent: 'ale również ludzi tu każdy swoją twoją wychodzić .'
ic| tgt_sent: 'są też jednak bardzo skuteczne w naszej uwagi .'


ic| src_sent: 'auta a - mluvil jsem o nich v prezentaci , ale chci vám to ukázat v kontextu .'
ic| pred_sent: 'i i - widziałem ze sobą w londynie , ale masz musimy to tego w stanie .'
ic| tgt_sent: ('samochody i ciężarówki - mówiłem o tym przy , ale chcę na to z szerszej '
               'perspektywy .')


ic| src_sent: ('je to jednoduchý , zřejmý cíl našich obav , a měl by být , ale znečištění '
               'související s budovami je pro globální oteplování horší , než z aut a .')
ic| pred_sent: ('jest pan sugeruje , moje urodziny dla firmy , i po nasz dom , a kto do do '
                "którego jesteś członkiem ' , od od ?")
ic| tgt_sent: ('to jasny i widoczny problem , i tak powinno być , ale więcej zanieczyszczeń '
               'globalne pochodzi z budynków niż z samochodów i ciężarówek .')


ic| src_sent: 'auta a jsou velmi významné a máme nejnižší standardy na světě ,'
ic| pred_sent: 'a i ten tu zamek i wiem w rodzaju na podłodze ,'
ic| tgt_sent: ('samochody osobowe i ciężarówki mają istotne znaczenie , a my mamy najniższe '
               'normy na świecie ,')
  0%|                                                                                                                                                                                                                                                                | 1/2359 [00:41<27:05:16, 41.36s/it]